#Árvore de decisão para atributos contínuos

In [ ]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y=True)

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
import numpy as np
from sklearn.model_selection import cross_validate

def maisFrequente(y):
  return Counter(y.flat).most_common(1)[0][0]

class ZeroR(BaseEstimator, ClassifierMixin):
  def fit(self, X, y):
    self.resposta = maisFrequente(y)
    return self
  def predict(self, X, y=None):
    y = np.empty((X.shape[0]))
    y[:] = self.resposta
    return y

scores = cross_validate(ZeroR(), X, y)
scores['test_score'], np.mean(scores['test_score'])

(array([0.40350877, 0.57017544, 0.64912281, 0.74561404, 0.7699115 ]),
 0.6276665114112715)

In [ ]:
modelo = ZeroR()
modelo.fit(X, y)
modelo.resposta

1

In [ ]:
modelo = ZeroR()
modelo.fit(X, y)
ypred = modelo.predict(X)
sum(y==ypred)/len(y)

0.6274165202108963

In [ ]:
class Arvore(BaseEstimator, ClassifierMixin):
  def fit(self, X, y):
    self.caracteristica = 2
    self.valor = np.mean(X[:,self.caracteristica])
    maiores = X[:,self.caracteristica] > self.valor
    if sum(maiores)>0 and sum(~maiores)>0:
      self.maiores = Arvore()
      self.maiores.fit(X[maiores,:], y[maiores])
      self.menores = Arvore()
      self.menores.fit(X[~maiores,:], y[~maiores])
    else:
      self.resposta = maisFrequente(y)
    return self
  def predict(self, X, y=None):
    y = np.empty((X.shape[0]))
    if hasattr(self, 'resposta'):
      y[:] = self.resposta
    else:
      maiores = X[:,self.caracteristica] > self.valor
      y[maiores] = self.maiores.predict(X[maiores,:])
      y[~maiores] = self.menores.predict(X[~maiores,:])
    return y

modelo = Arvore()
modelo.fit(X, y)
ypred = modelo.predict(X)
sum(y==ypred)/len(y)

0.9876977152899824

In [ ]:
scores = cross_validate(Arvore(), X, y)
scores['test_score'], np.mean(scores['test_score'])

(array([0.77192982, 0.79824561, 0.86842105, 0.81578947, 0.82300885]),
 0.8154789628939605)

In [ ]:
def impureza(y): #Gini
  labels = list(set(y))
  labels.sort()
  probabilidades = np.zeros((len(labels),))
  for i, k in enumerate(labels):
    probabilidades[i] = sum(y==k)/len(y)
  result = 1 - sum(probabilidades ** 2)
  return result

impureza(y[:])

0.4675300607546925

In [ ]:
def impurezaValor(x, y, valor):
  maiores = x > valor
  impurezamaiores = impureza(y[maiores])
  proporcaomaiores = sum(maiores)/len(y)
  impurezamenores = impureza(y[~maiores])
  proporcaomenores = sum(~maiores)/len(y)
  impurezaTotal = proporcaomaiores*impurezamaiores + proporcaomenores*impurezamenores
  return impurezaTotal, impurezamenores, impurezamaiores

impurezaValor(X[:,2], y, 2.5)

(0.4675300607546925, 1, 0.4675300607546925)

In [ ]:
def melhorValor(x, y):
  result = None
  menorImpureza = float('inf')
  xmax = np.max(x)
  xmin = np.min(x)
  while True:
    valor = (xmin+xmax)/2
    impTotal, impMenores, impMaiores = impurezaValor(x, y, valor)
    if impTotal < menorImpureza:
      menorImpureza = impTotal
      result = valor
      if impMaiores == 0 or impMenores == 0:
        break
      if impMaiores < impMenores:
        xmin = valor
      else:
        xmax = valor
    else:
      break
  return result, menorImpureza

melhorValor(X[:,2], y)

(116.145, 0.30371755118489246)

In [ ]:
def melhorCaracteristica(X, y):
  impurezas = []
  valores = []
  for caracteristica in range(X.shape[1]):
    valor, imp = melhorValor(X[:,caracteristica], y)
    impurezas.append(imp)
    valores.append(valor)
  # print(impurezas)
  # print(valores)
  impurezas = np.array(impurezas)
  caracteristica = np.argmin(impurezas)
  return impurezas[caracteristica], caracteristica, valores[caracteristica]

melhorCaracteristica(X, y)

(0.1504971683890802, 27, 0.1455)

In [ ]:
class Arvore(BaseEstimator, ClassifierMixin):
  def fit(self, X, y):
    self.impureza, self.caracteristica, self.valor = melhorCaracteristica(X, y)
    #print(f'{self.impureza}, {self.caracteristica}, {self.valor}')
    maiores = X[:,self.caracteristica] > self.valor
    if sum(maiores)>0 and sum(~maiores)>0:
      self.maiores = Arvore()
      self.maiores.fit(X[maiores,:], y[maiores])
      self.menores = Arvore()
      self.menores.fit(X[~maiores,:], y[~maiores])
    else:
      self.resposta = maisFrequente(y)
    return self
  def predict(self, X, y=None):
    y = np.empty((X.shape[0]))
    if hasattr(self, 'resposta'):
      y[:] = self.resposta
    else:
      maiores = X[:,self.caracteristica] > self.valor
      y[maiores] = self.maiores.predict(X[maiores,:])
      y[~maiores] = self.menores.predict(X[~maiores,:])
    return y

modelo = Arvore()
modelo.fit(X, y)
ypred = modelo.predict(X)
sum(y==ypred)/len(y)

1.0

In [ ]:
def melhorValorNova(x, y):
  result = None
  menorImpureza = float('inf')

  if x.shape[0] < 2:
    return x[0], 0.0

  x1 = np.sort(x)
  for i in range(y.shape[0] - 1):
    valor = (x1[i]+x1[i+1])/2
    impTotal, impMenores, impMaiores = impurezaValor(x1, y, valor)
    if impTotal < menorImpureza:
      menorImpureza = impTotal
      result = valor

  return result, menorImpureza

melhorValorNova(X[:,2], y)

(63.865, 0.4161477951424071)

In [ ]:
def melhorCaracteristicaNova(X, y):
  impurezas = []
  valores = []

  for caracteristica in range(X.shape[1]):
    valor, imp = melhorValorNova(X[:,caracteristica], y)
    impurezas.append(imp)
    valores.append(valor)

  impurezas = np.array(impurezas)
  caracteristica = np.argmin(impurezas)

  return impurezas[caracteristica], caracteristica, valores[caracteristica]

melhorCaracteristicaNova(X, y)

(0.4161477951424071, 0, 9.876)

In [ ]:
class ArvoreNova(BaseEstimator, ClassifierMixin):
  def fit(self, X, y):
    self.impureza, self.caracteristica, self.valor = melhorCaracteristicaNova(X, y)
    #print(f'fit: {X.shape[0]}, {self.impureza}, {self.caracteristica}, {self.valor}')
    maiores = X[:,self.caracteristica] > self.valor
    if sum(maiores)>0 and sum(~maiores)>0:
      self.maiores = ArvoreNova()
      self.maiores.fit(X[maiores,:], y[maiores])
      self.menores = ArvoreNova()
      self.menores.fit(X[~maiores,:], y[~maiores])
    else:
      self.resposta = maisFrequente(y)
    return self
  def predict(self, X, y=None):
    y = np.empty((X.shape[0]))
    if hasattr(self, 'resposta'):
      y[:] = self.resposta
    else:
      maiores = X[:,self.caracteristica] > self.valor
      y[maiores] = self.maiores.predict(X[maiores,:])
      y[~maiores] = self.menores.predict(X[~maiores,:])
    return y

modelo_novo = ArvoreNova()
modelo_novo.fit(X, y)
ypred = modelo_novo.predict(X)
sum(y==ypred)/len(y)

1.0

In [ ]:
scores = cross_validate(Arvore(), X, y)
scores['test_score'], np.mean(scores['test_score'])

(array([0.89473684, 0.92982456, 0.93859649, 0.93859649, 0.94690265]),
 0.9297314081664337)

In [ ]:
scores = cross_validate(ArvoreNova(), X, y)
scores['test_score'], np.mean(scores['test_score'])

(array([0.78070175, 0.79824561, 0.90350877, 0.85087719, 0.80530973]),
 0.8277286135693215)

O Score Médio da heurística original apresenta melhor desempenho

In [ ]:
import timeit
modelo = Arvore()
result = timeit.timeit(stmt='modelo.fit(X, y)', globals=globals(), number=10)
print('Arvore (tempo de execução):', result)

Arvore (tempo de execução): 18.01477430199975


In [ ]:
import timeit
modelo_novo = ArvoreNova()
result = timeit.timeit(stmt='modelo_novo.fit(X, y)', globals=globals(), number=10)
print('ArvoreNova (tempo de execução):', result)

ArvoreNova (tempo de execução): 473.18973948800067


A diferença de tempo foi significativa entre as 2 heurísticas, sendo a original muita mais rápida
